This script merges ORF candidates from all experiments into one table in the CSV format.
The resulting table will be stored as {local_folder}/merged_orfs_found_by_any_caller.csv
This is the first step of the ORF score prediction and only need to be executed once when you update the experiment list.

In [4]:
import pandas as pd
import subprocess
import shlex
import os

In [5]:
def download_feature_set(experiment_name, local_path):
    orf_path = f"s3://velia-piperuns-dev/{experiment_name}/output/{experiment_name}_found_by_any_caller.csv"
    local_path = f"{local_folder}/{experiment_name}_found_by_any_caller.csv"
    cmd = f"aws s3 cp {orf_path} {local_path}"
    print(cmd)
    subprocess.run(shlex.split(cmd))
    return local_path

def load_orfs(experiment_name, local_folder, merged_orfs_info, orf_callers,
              orfs_to_skip, annotations_to_exclude):
    data_path = download_feature_set(experiment_name, local_folder)
    if not os.path.exists(data_path):
        return 0
    
    orfs_info = {}
    for index, row in pd.read_csv(data_path, sep='\t').iterrows():
        orf_key = (row["chrom_id"], row["orf_start"], row["orf_end"], row["strand"], row["exon_blocks"])
        if orf_key not in merged_orfs_info:
            merged_orfs_info[orf_key] = row
            merged_orfs_info[orf_key] = pd.concat([merged_orfs_info[orf_key],
                                                   pd.Series({"#called": 1,
                                                              "cell_types": experiment_name})])
        else:
            merged_orfs_info[orf_key]["#called"] += 1
            merged_orfs_info[orf_key]["cell_types"] += ("," + experiment_name)
            for orf_caller in orf_callers:
                if (type(row[f"orf_score_{orf_caller}"]) == type("")
                    and (not row[f"orf_score_{orf_caller}"].strip())):
                    continue
                else:
                    if ((type(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) == type("") and
                         (not merged_orfs_info[orf_key][f"orf_score_{orf_caller}"].strip()))
                        or (merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] is None)
                        or (float(merged_orfs_info[orf_key][f"orf_score_{orf_caller}"]) <= float(row[f"orf_score_{orf_caller}"]))):
                        merged_orfs_info[orf_key][f"orf_score_{orf_caller}"] = row[f"orf_score_{orf_caller}"]
                        merged_orfs_info[orf_key][f"ORF_type_{orf_caller}"] = row[f"ORF_type_{orf_caller}"]
                        merged_orfs_info[orf_key][f"gene_id_{orf_caller}"] = row[f"gene_id_{orf_caller}"]
                        merged_orfs_info[orf_key][f"transcript_id_{orf_caller}"] = row[f"transcript_id_{orf_caller}"]
                        if row[f"ORF_type_{orf_caller}"] in annotations_to_exclude:
                            orfs_to_skip.add(orf_key)
    #os.remove(data_path)
    return 1

In [ ]:
local_folder= "./data"
orf_callers = ["price", "ribotish", "ribocode"]

# load ORFs from each experiment
merged_orfs_info = {}
orfs_to_skip = set()
count = 0
annotations_to_exclude = ["Trunc", "Ext", "Truncated", "Extended"]
#### TODO(steve): modify the list above to exclude ORFs of certain types.
# ORF types to exclude:
# PRICE: "Trunc", "Ext"
# ribotish: "Truncated", "Extended"
# ribocode: no obivious appliable types
for experiment_name in open(f"{local_folder}/experiments.txt"):
    experiment_name = experiment_name.split("PRE")[1].strip()[:-1]
    count += load_orfs(experiment_name, local_folder, merged_orfs_info, orf_callers,
                       orfs_to_skip, annotations_to_exclude)
print("{count} datasets are loaded.")

# remove orfs with annotations listed in annotations_to_exclude
for orf_key in orfs_to_skip:
    del merged_orfs_info[orf_key]
print(f"{len(orfs_to_skip)} ORFs have been deleted.")
    
# save the merged ORFs into file
# this file is saved to a local folder and needs to be uploaded to AWS manually
pd.DataFrame(merged_orfs_info.values()).reset_index().to_csv(
    f"{local_folder}/merged_orfs_found_by_any_caller_filtered.csv",
    sep="\t",
    index=False)

aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1A/output/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1A/output/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_1A_found_by_any_caller.csv
aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1B/output/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_1B/output/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_1B_found_by_any_caller.csv


/tmp/ipykernel_62869/1228207664.py:16: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2A/output/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2A/output/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_2A_found_by_any_caller.csv


/tmp/ipykernel_62869/1228207664.py:16: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  for index, row in pd.read_csv(data_path, sep='\t').iterrows():


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2B/output/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv ./data/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv
download: s3://velia-piperuns-dev/VPR_orfcalling_20240307222145_MB1_2B/output/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv to data/VPR_orfcalling_20240307222145_MB1_2B_found_by_any_caller.csv


...

aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240308012940_SRX876073/output/VPR_orfcalling_20240308012940_SRX876073_found_by_any_caller.csv ./data/VPR_orfcalling_20240308012940_SRX876073_found_by_any_caller.csv


fatal error: An error occurred (404) when calling the HeadObject operation: Key "VPR_orfcalling_20240308012940_SRX876073/output/VPR_orfcalling_20240308012940_SRX876073_found_by_any_caller.csv" does not exist


aws s3 cp s3://velia-piperuns-dev/VPR_orfcalling_20240308012940_SRX876074/output/VPR_orfcalling_20240308012940_SRX876074_found_by_any_caller.csv ./data/VPR_orfcalling_20240308012940_SRX876074_found_by_any_caller.csv


fatal error: An error occurred (404) when calling the HeadObject operation: Key "VPR_orfcalling_20240308012940_SRX876074/output/VPR_orfcalling_20240308012940_SRX876074_found_by_any_caller.csv" does not exist


{count} datasets are loaded.
57933 ORFs have been deleted.


In [7]:
df = pd.read_csv('data/merged_orfs_found_by_any_caller_filtered.csv', sep='\t')

In [9]:
set(df['cell_types'])

{'VPR_orfcalling_20240307222145_MB1_1A',
 'VPR_orfcalling_20240307222145_MB1_1A,VPR_orfcalling_20240307222145_MB1_1B',
 'VPR_orfcalling_20240307222145_MB1_1A,VPR_orfcalling_20240307222145_MB1_1B,VPR_orfcalling_20240307222145_MB1_2A',
 'VPR_orfcalling_20240307222145_MB1_1A,VPR_orfcalling_20240307222145_MB1_1B,VPR_orfcalling_20240307222145_MB1_2A,VPR_orfcalling_20240307222145_MB1_2B',
 'VPR_orfcalling_20240307222145_MB1_1A,VPR_orfcalling_20240307222145_MB1_1B,VPR_orfcalling_20240307222145_MB1_2B',
 'VPR_orfcalling_20240307222145_MB1_1A,VPR_orfcalling_20240307222145_MB1_2A',
 'VPR_orfcalling_20240307222145_MB1_1A,VPR_orfcalling_20240307222145_MB1_2A,VPR_orfcalling_20240307222145_MB1_2B',
 'VPR_orfcalling_20240307222145_MB1_1A,VPR_orfcalling_20240307222145_MB1_2B',
 'VPR_orfcalling_20240307222145_MB1_1B',
 'VPR_orfcalling_20240307222145_MB1_1B,VPR_orfcalling_20240307222145_MB1_2A',
 'VPR_orfcalling_20240307222145_MB1_1B,VPR_orfcalling_20240307222145_MB1_2A,VPR_orfcalling_20240307222145_MB1